# Vérification de la version de python

In [1]:
import sys
print(sys.version)

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]


# Importation des packages

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

# Pour la récupération de la base de données data.gouv.fr
import requests
import tempfile
import zipfile

# Définition de fonctions

- Fonction récupérée sur la page web **(1)** du paragraphe **Sources**

In [3]:
def download_unzip(url, dirname = tempfile.gettempdir(), destname = "borders"):
  myfile = requests.get(url)
  open("{}/{}.zip".format(dirname, destname), 'wb').write(myfile.content)
  with zipfile.ZipFile("{}/{}.zip".format(dirname, destname), 'r') as zip_ref:
      zip_ref.extractall(dirname + '/' + destname)

# Importation des données

### Base de données *Atout France*

In [4]:
BDD = pd.read_csv("Base de données Atout France.csv", sep = ";")
BDD.head()

,CODGEO,POP18,Lits marchands 2018 (hors PtoP),Lits touristiques 2021 inc. PtoP,P18_RSECOCC,Lits marchands 2015 (hors PtoP),P15_RSECOCC,LIBGEO
0,6001,93,0,296.000000,56.000000,64,51.696721,Aiglun
1,6002,60,0,109.000000,21.000000,0,17.000000,Amirat
2,6003,655,24,2580.000000,500.000000,172,459.280576,Andon
3,6004,74047,10517,145083.979374,22825.595875,13876,21090.166718,Antibes
4,6005,172,0,599.638227,113.327645,0,125.529661,Ascros


In [5]:
BDD.shape # Dimension du tableau

(3071, 8)

### Base de données du découpage administratif communal français issu d'OpenStreetMap (*data.gouv.fr*)

On utilise cette base pour récupérer les latitudes et longitudes de chaque commune française.

**A savoir :** Depuis le 1er janvier 2022, les communes Saint-Gérand et Croixanvec sont des communes déléguées de la commune nouvelle de Saint-Gérand-Croixanvec **(2)**. Ces communes ne font donc plus partie de la base de données actuelle de *data.gouv.fr* mais font partie néanmoins de notre base de données Atout France. On choisit donc de prendre l'avant dernière version de la base *data.gouv.fr* (01/01/2021) afin de garder ces deux communes.

In [6]:
url = 'https://www.data.gouv.fr/fr/datasets/r/17062524-991f-4e13-9bf0-b410cc2216fd'

temporary_location = tempfile.gettempdir()

download_unzip(url)
communes = gpd.read_file(temporary_location + "/borders/communes-20210101.shp")

In [7]:
communes.head()

,insee,nom,wikipedia,surf_ha,geometry
0,30225,Sabran,fr:Sabran (Gard),3563.0,"POLYGON ((4.49703 44.14344, 4.49727 44.14398, ..."
1,26334,Salettes,fr:Salettes (Drôme),706.0,"POLYGON ((4.94213 44.57481, 4.94283 44.57501, ..."
2,04200,Salignac,fr:Salignac (Alpes-de-Haute-Provence),1435.0,"POLYGON ((5.95337 44.15613, 5.95341 44.15649, ..."
3,04179,Saint-Geniez,fr:Saint-Geniez,3900.0,"POLYGON ((5.98593 44.23280, 5.98654 44.23292, ..."
4,05162,La Saulce,fr:La Saulce,771.0,"POLYGON ((5.98920 44.41892, 5.98921 44.41894, ..."


In [8]:
communes.shape

(34969, 5)

# Manipulation des données

### Base *Atout France*

Tout d'abord, on remarque que certains codes INSEE (colonne *CODGEO*) comportent seulement 4 chiffres, à savoir les codes INSEE commençant normalement par "0". On va donc les rajouter et considérer la colonne *CODGEO* comme une liste de caractères et non d'entiers.

In [9]:
BDD["CODGEO"] = BDD["CODGEO"].apply(lambda x: str(x).zfill(5))

In [10]:
BDD.head()

,CODGEO,POP18,Lits marchands 2018 (hors PtoP),Lits touristiques 2021 inc. PtoP,P18_RSECOCC,Lits marchands 2015 (hors PtoP),P15_RSECOCC,LIBGEO
0,06001,93,0,296.000000,56.000000,64,51.696721,Aiglun
1,06002,60,0,109.000000,21.000000,0,17.000000,Amirat
2,06003,655,24,2580.000000,500.000000,172,459.280576,Andon
3,06004,74047,10517,145083.979374,22825.595875,13876,21090.166718,Antibes
4,06005,172,0,599.638227,113.327645,0,125.529661,Ascros


Ensuite, on va changer le nom des variables pour faciliter les manipulations de données qui suivront.

In [11]:
BDD.columns

Index(['CODGEO', 'POP18', 'Lits marchands 2018 (hors PtoP)',
       'Lits touristiques 2021 inc. PtoP', 'P18_RSECOCC',
       'Lits marchands 2015 (hors PtoP)', 'P15_RSECOCC', 'LIBGEO'],
      dtype='object')

In [12]:
BDD = BDD.rename(columns={"CODGEO": "insee",
                    'LIBGEO': "nom2",
                    "POP18": "POP18",
                    'Lits marchands 2015 (hors PtoP)': "marchands15",
                    'Lits marchands 2018 (hors PtoP)': "marchands18",
                    'Lits touristiques 2021 inc. PtoP': "touristiques21",
                    "P15_RSECOCC": "P15_RSECOCC",
                    'P18_RSECOCC': 'P18_RSECOCC'})

In [13]:
BDD.head()

,insee,POP18,marchands18,touristiques21,P18_RSECOCC,marchands15,P15_RSECOCC,nom2
0,06001,93,0,296.000000,56.000000,64,51.696721,Aiglun
1,06002,60,0,109.000000,21.000000,0,17.000000,Amirat
2,06003,655,24,2580.000000,500.000000,172,459.280576,Andon
3,06004,74047,10517,145083.979374,22825.595875,13876,21090.166718,Antibes
4,06005,172,0,599.638227,113.327645,0,125.529661,Ascros


Enfin, on va créer les informations que l'on souhaite représenter pour les cartographies 2 et 3, à savoir le taux de fonction touristique en 2018 et le taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018.

D'après le site *Géoconfluences* **(3)**, le taux de fonction touristique désigne le rapport entre la capacité d’une commune en hébergement touristique (c’est-à-dire le nombre de touristes pouvant être accueillis) et le nombre de résidents permanents.

Pour réaliser ce calcul, on a à notre disposition le nombre de lits marchands en 2018 (*marchands18*), le nombre de résidences secondaires et logements occasionnels en 2018 (*P18_RSECOCC*) **(4)** et le nombre d'habitants en 2018 (*POP18*).

D'après les documents **(5)** et **(6)**, on considère que l'on a 5 lits par résidence secondaire quelle que soit la taille du logement et que le nombre de lits touristiques est égale au nombre de lits marchands (banalisés) plus le nombre de lits non marchands, c'est-à-dire indisponibles à la location : les résidences secondaires.

In [14]:
BDD["touristiques18"] = BDD["marchands18"] + 5*BDD["P18_RSECOCC"]
BDD["taux_fonction_touristique18"] = round(BDD["touristiques18"]/BDD["POP18"], 2)

Le taux de croissance annuel moyen (TCAM) permet de calculer la croissance économique d’une date (*v_year0*) à une autre date (*v_year1*) avec *n* le nombre d'années de la période.

Voici la formule du TCAM **(7)**: $$TCAM = (\sqrt[n]{\frac{v\_year1}{v\_year0}}-1)\times 100$$

In [15]:
BDD["TCAM_15_18"] = round(((BDD["marchands18"]/BDD["marchands15"])**(1/(2018-2015))-1)*100,2)

In [16]:
BDD.head()

,insee,POP18,marchands18,touristiques21,P18_RSECOCC,marchands15,P15_RSECOCC,nom2,touristiques18,taux_fonction_touristique18,TCAM_15_18
0,06001,93,0,296.000000,56.000000,64,51.696721,Aiglun,280.000000,3.01,-100.00
1,06002,60,0,109.000000,21.000000,0,17.000000,Amirat,105.000000,1.75,NaN
2,06003,655,24,2580.000000,500.000000,172,459.280576,Andon,2524.000000,3.85,-48.13
3,06004,74047,10517,145083.979374,22825.595875,13876,21090.166718,Antibes,124644.979374,1.68,-8.82
4,06005,172,0,599.638227,113.327645,0,125.529661,Ascros,566.638227,3.29,NaN


On remarque que dans la colonne *TCAM_15_18*, certaines cases ont pour valeurs *NaN* ou *inf*. Cela est dû à cause de la division par zéro.

Pour l'analyse cartographique de ce taux, on va considérer que si le nombre de lits marchands de 2015 **ET** le nombre de lits marchands de 2018 sont nuls, alors le TCAM de l'hébergement marchands entre 2015 et 2018 est nul également, il n'y a pas eu de croissance.

Cependant si la valeur du TCAM vaut *inf* (cas où seulement le nombre de lits marchands de 2015 est nul), on va considérer la valeur du TCAM comme *NaN* et donc supprimer cette information de la table de données (plus tard dans le point Cartographie n°3) car cela n'a pas de sens de parler d'une croissance économique infinie.

In [17]:
BDD.loc[((BDD["marchands18"]==0) & (BDD["marchands15"]==0)), "TCAM_15_18"] = 0.0

In [18]:
BDD.head()

,insee,POP18,marchands18,touristiques21,P18_RSECOCC,marchands15,P15_RSECOCC,nom2,touristiques18,taux_fonction_touristique18,TCAM_15_18
0,06001,93,0,296.000000,56.000000,64,51.696721,Aiglun,280.000000,3.01,-100.00
1,06002,60,0,109.000000,21.000000,0,17.000000,Amirat,105.000000,1.75,0.00
2,06003,655,24,2580.000000,500.000000,172,459.280576,Andon,2524.000000,3.85,-48.13
3,06004,74047,10517,145083.979374,22825.595875,13876,21090.166718,Antibes,124644.979374,1.68,-8.82
4,06005,172,0,599.638227,113.327645,0,125.529661,Ascros,566.638227,3.29,0.00


### Base *data.gouv.fr*

On décide de ne garder que les colonnes suivantes :
- *insee*
- *nom*
- *geometry*

In [19]:
communes.columns

Index(['insee', 'nom', 'wikipedia', 'surf_ha', 'geometry'], dtype='object')

In [20]:
communes = communes.drop(columns=['wikipedia', 'surf_ha'])
communes.head()

,insee,nom,geometry
0,30225,Sabran,"POLYGON ((4.49703 44.14344, 4.49727 44.14398, ..."
1,26334,Salettes,"POLYGON ((4.94213 44.57481, 4.94283 44.57501, ..."
2,04200,Salignac,"POLYGON ((5.95337 44.15613, 5.95341 44.15649, ..."
3,04179,Saint-Geniez,"POLYGON ((5.98593 44.23280, 5.98654 44.23292, ..."
4,05162,La Saulce,"POLYGON ((5.98920 44.41892, 5.98921 44.41894, ..."


# Cartographie n°1 : Nombre d’habitants en 2018

On décide de créer un nouveau dataframe, *df1*, en fusionnant les dataframes *BDD* (où on ne garde que les colonnes *insee* et *POP18*) et *communes*.

In [21]:
df1 = BDD.drop(columns = ['marchands18', 'touristiques21', 'P18_RSECOCC',
                          'marchands15', 'P15_RSECOCC', 'nom2', 'touristiques18',
                          'taux_fonction_touristique18', 'TCAM_15_18'])


df1 = gpd.GeoDataFrame(df1).merge(communes, on='insee')

En rangeant la colonne *POP18* par ordre croissant, on remarque que la commune avec le plus d'habitants est **Argol** avec 2161000 habitants. En faisant des recherches **(8)**, on trouve que cette commune n'avait même pas 1000 habitants recensés en 2019. On suppose donc qu'il y a une erreur dans le jeu de données et on décide donc de supprimer cette ligne du jeu de données.

In [22]:
df1.sort_values("POP18").tail()

,insee,POP18,nom,geometry
1312,33063,260352,Bordeaux,"POLYGON ((-0.63870 44.85916, -0.63827 44.85994..."
1957,34172,293410,Montpellier,"POLYGON ((3.80706 43.63465, 3.80707 43.63469, ..."
2559,44109,319284,Nantes,"POLYGON ((-1.64181 47.19116, -1.64122 47.19121..."
87,06088,343889,Nice,"POLYGON ((7.18195 43.73746, 7.18195 43.73764, ..."
974,29001,2161000,Argol,"MULTIPOLYGON (((-4.36723 48.27782, -4.36650 48..."


In [23]:
df1 = df1.drop(df1[df1.insee=="29001"].index)

df1[df1.insee=="29001"] # Vérification si la commune Argol a bien été supprimée

,insee,POP18,nom,geometry


Pour lire la carte plus facilement, on décide d'ajouter de la couleur en fonction du nombre d'habitants (palette nuancée du jaune au rouge où le jaune représente le moins d'habitants et le rouge le plus d'habitants).

Pour réaliser cela, on crée une nouvelle colonne, *bins_POP18*, qui segmente et trier les valeurs de données de la colonne *POP18* dans des "bins" 

In [24]:
# Création colonne bins_POP18
bins_POP18 = pd.IntervalIndex.from_tuples([(0, 250), (250, 500), (500, 750), (750, 1000), (1000, 5000),
                                           (5000, 10000), (10000, 50000), (50000, 100000), (100000, float("inf"))])
                                        # Ici, les valeurs des intervalles sont choisies arbitrairement

bins_labels_mapping_POP18 = dict(zip(bins_POP18, range(1, len(bins_POP18) + 1)))

bins_POP18 = pd.cut(df1['POP18'], bins=bins_POP18, retbins=False).map(bins_labels_mapping_POP18).astype(float)

df1['bins_POP18'] = bins_POP18

# Création palette
palette_POP18 = {
    'name': 'palette_POP18',
    'type': 'custom',
    'category': 'Custom',
    'colors': ['#FFFFCC', '#FFEDA0', '#FED976', '#FEB24C', '#FD8D3C', '#FC4E2A', '#E31A1C', '#BD0026', '#800026']}

In [25]:
df1.head()

,insee,POP18,nom,geometry,bins_POP18
0,06001,93,Aiglun,"POLYGON ((6.90718 43.86292, 6.90718 43.86315, ...",1.0
1,06002,60,Amirat,"POLYGON ((6.78127 43.88345, 6.78129 43.88348, ...",1.0
2,06003,655,Andon,"POLYGON ((6.76869 43.80453, 6.76870 43.80460, ...",3.0
3,06004,74047,Antibes,"MULTIPOLYGON (((7.06442 43.60335, 7.06539 43.6...",8.0
4,06005,172,Ascros,"POLYGON ((6.97236 43.92248, 6.97329 43.92298, ...",1.0


On crée ensuite l'objet cartographie et on ajoute les données à cet objet

In [26]:
carto = KeplerGl()
carto.add_data(data=df1.to_json(), name = "Nombre d'habitants en 2018")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


L'objet *carto* doit afficher une carte intéractive.

**A savoir** : Après avoir lancé le chunk suivant, on le met en commentaire et on relance le chunk pour éviter de surcharger le notebook.

In [31]:
# carto

On regarde la configuration par défaut de la cartographie.


**A savoir** : Si le résultat du chunk suivant est un dictionnaire vide, relancer le chunk précédent jusqu'à ce que le dictionnaire ne soit plus vide.

In [28]:
carto.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '59lh7ho',
     'type': 'geojson',
     'config': {'dataId': "Nombre d'habitants en 2018",
      'label': "Nombre d'habitants en 2018",
      'color': [18, 147, 154],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [221, 178, 124],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'si

On change la configuration par défaut de la carte pour avoir une visualisation parlante.

In [29]:
tmp_config = carto.config

layers = tmp_config['config']['visState']['layers']
layer_to_update = [layer for layer in layers if layer['config']['dataId'] == "Nombre d'habitants en 2018"]
layer_to_update = layer_to_update[0]

# Pour signifier que le remplissage des communes se fait en fonction de la colonne bins_POP18
layer_to_update["visualChannels"].update({"colorField": {"name": "bins_POP18", "type": "integer"},
                                          "colorScale": "quantize",})

# Pour changer la palette de couleurs remplissant les communes
layer_to_update["config"]["visConfig"]["colorRange"].update(palette_POP18)

# Pour changer l'épaisseur des frontières
layer_to_update["config"]["visConfig"]["thickness"] = 0.2

# Pour changer la couleur des frontières ([0, 0, 0] réprésente la couleur noire en RGB)
layer_to_update["config"]["visConfig"]["strokeColor"] = [0, 0, 0]

carto.config = tmp_config

On enregistre la cartographie au format HTML.

In [30]:
carto.save_to_html(file_name='./Nombre d’habitants en 2018.html')

Map saved to ./Nombre d’habitants en 2018.html!


# Cartographie n°2 : Taux de fonction touristique du Finistère de 2018

On décide de créer un nouveau dataframe, *df2*, en fusionnant les dataframes *BDD* (où on ne garde que les colonnes *insee* et *taux_fonction_touristique18*) et *communes*. Comme on s'intéresse seulement au Finistère, on ne garde que les codes INSEE commençant par 29. 

In [32]:
df2 = BDD.drop(columns = ['marchands18', 'touristiques21', 'P18_RSECOCC',
                          'marchands15', 'P15_RSECOCC', 'nom2', 'touristiques18',
                          'POP18', 'TCAM_15_18'])

df2 = df2[df2.insee.str.startswith('29')]

df2 = gpd.GeoDataFrame(df2).merge(communes, on='insee')

On rencontre le même problème que dans la section *Cartographie n°1* car **Argol** est une commune du Finistère. On décide donc de supprimer cette ligne du jeu de données.

In [33]:
df2 = df2.drop(df2[df2.insee=="29001"].index)

df2[df2.insee=="29001"] # Vérification si la commune Argol a bien été supprimée

,insee,taux_fonction_touristique18,nom,geometry


Pour lire la carte plus facilement, on décide d'ajouter de la couleur en fonction du taux de fonction touristique (palette nuancée du vert clair au vert foncé où le vert clair représente un taux faible et le vert foncé un taux élevé).

Pour réaliser cela, on crée une nouvelle colonne, *bins_TFT18*, qui segmente et trier les valeurs de données de la colonne *taux_fonction_touristique18* dans des "bins".

A l'aide du document **(9)**, on prend la décision de répartir en 5 classes les communes du Finistère selon leur taux de fonction touristique.

Deux classes de communes dites « à faible intensité touristique » :
- classe 1 : taux de fonction touristique inférieur à 50 lits pour 100 habitants ;  
- classe 2 : taux de fonction touristique compris entre 50 et 100 lits pour 100 habitants.

Trois classes de communes, dites « à forte intensité touristique » (susceptibles d’au moins doubler leur
population) :
- classe 3 : taux de fonction touristique compris entre 100 et 200 lits pour 100 habitants ;  
- classe 4 : taux de fonction touristique compris entre 200 et 1 000 lits pour 100 habitants ;  
- classe 5 : taux de fonction touristique supérieur ou égal à 1 000 lits pour 100 habitants.

L'unité de la colonne *taux_fonction_touristique18* est actuellement en lits par habitant.

In [34]:
# Création colonne bins_TFT18
bins_TFT18 = pd.IntervalIndex.from_tuples([(0, 0.5), (0.5, 1.0), (1.0, 2.0), (2.0, 10.0), (10.0, float("inf"))])
                                          # Ici, les valeurs des intervalles sont choisies d'après le document (9)

bins_labels_mapping_TFT18 = dict(zip(bins_TFT18, range(1, len(bins_TFT18) + 1)))

bins_TFT18 = pd.cut(df2['taux_fonction_touristique18'],
                    bins=bins_TFT18, retbins=False).map(bins_labels_mapping_TFT18).astype(float)

df2['bins_TFT18'] = bins_TFT18

# Création palette
palette_TFT18 = {
    'name': 'palette_TFT18',
    'type': 'custom',
    'category': 'Custom',
    'colors': ['#addd8e', '#78c679', '#41ab5d', '#238443', '#006837']}

In [35]:
df2.head()

,insee,taux_fonction_touristique18,nom,geometry,bins_TFT18
1,29002,0.13,Arzano,"POLYGON ((-3.50348 47.89572, -3.50334 47.89597...",1.0
2,29003,2.61,Audierne,"POLYGON ((-4.60262 48.01621, -4.60257 48.01686...",4.0
3,29004,0.16,Bannalec,"POLYGON ((-3.76427 47.92862, -3.76424 47.92912...",1.0
4,29005,0.31,Baye,"POLYGON ((-3.63388 47.84362, -3.63286 47.84428...",1.0
5,29006,5.38,Bénodet,"POLYGON ((-4.11753 47.87927, -4.11749 47.87936...",4.0


On crée ensuite l'objet cartographie et on ajoute les données à cet objet

In [36]:
carto = KeplerGl()
carto.add_data(data=df2.to_json(), name="Taux de fonction touristique du Finistère de 2018")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


L'objet *carto* doit afficher une carte intéractive.

**A savoir** : Après avoir lancé le chunk suivant, on le met en commentaire et on relance le chunk pour éviter de surcharger le notebook.

In [41]:
# carto

On regarde la configuration par défaut de la cartographie.


**A savoir** : Si le résultat du chunk suivant est un dictionnaire vide, relancer le chunk précédent jusqu'à ce que le dictionnaire ne soit plus vide.

In [38]:
carto.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '6dp5rw',
     'type': 'geojson',
     'config': {'dataId': 'Taux de fonction touristique du Finistère de 2018',
      'label': 'Taux de fonction touristique du Finistère de 2018',
      'color': [136, 87, 44],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [255, 153, 31],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '

On change la configuration par défaut de la carte pour avoir une visualisation parlante.

In [39]:
tmp_config = carto.config

layers = tmp_config['config']['visState']['layers']
layer_to_update = [layer for layer in layers if layer['config']['dataId'] == "Taux de fonction touristique du Finistère de 2018"]
layer_to_update = layer_to_update[0]

# Pour signifier que le remplissage des communes se fait en fonction de la colonne bins_TFT18
layer_to_update["visualChannels"].update({"colorField": {"name": "bins_TFT18", "type": "integer"},
                                          "colorScale": "quantize",
                                         })

# Pour changer la palette de couleurs remplissant les communes
layer_to_update["config"]["visConfig"]["colorRange"].update(palette_TFT18)

# Pour changer l'épaisseur des frontières
layer_to_update["config"]["visConfig"]["thickness"] = 0.2

# Pour changer la couleur des frontières ([0, 0, 0] réprésente la couleur noire en RGB)
layer_to_update["config"]["visConfig"]["strokeColor"] = [0, 0, 0]

carto.config = tmp_config

On enregistre la cartographie au format HTML.

In [40]:
carto.save_to_html(file_name='./Taux de fonction touristique du Finistère de 2018.html')

Map saved to ./Taux de fonction touristique du Finistère de 2018.html!


# Cartographie n°3 : TCAM de l’hébergement marchand du Finistère entre 2015 et 2018

On décide de créer un nouveau dataframe, *df3*, en fusionnant les dataframes *BDD* (où on ne garde que les colonnes *insee* et *TCAM_15_18*) et *communes*. Comme on s'intéresse seulement au Finistère, on ne garde que les codes INSEE commençant par 29. 

In [60]:
df3 = BDD.drop(columns = ['marchands18', 'touristiques21', 'P18_RSECOCC',
                          'marchands15', 'P15_RSECOCC', 'nom2', 'touristiques18',
                          'POP18', 'taux_fonction_touristique18'])

df3 = df3[df3.insee.str.startswith('29')]

df3 = gpd.GeoDataFrame(df3).merge(communes, on='insee')

On rencontre le même problème que dans la section *Cartographie n°1* car **Argol** est une commune du Finistère. On décide donc de supprimer cette ligne du jeu de données.

In [61]:
df3 = df3.drop(df3[df3.insee=="29001"].index)

df3[df3.insee=="29001"] # Vérification si la commune Argol a bien été supprimée

,insee,TCAM_15_18,nom,geometry


On transforme les valeurs inf en NA et on supprime les lignes où la valeur de la colonne *TCAM_15_18* est un NA.

In [62]:
df3[df3["TCAM_15_18"]==max(df3["TCAM_15_18"])]

,insee,TCAM_15_18,nom,geometry
199,29218,inf,Pont-Croix,"POLYGON ((-4.53940 48.04911, -4.53928 48.05001..."


In [63]:
df3 = df3.replace([np.inf, -np.inf], np.nan)\
.dropna(subset=["TCAM_15_18"], how="all")

In [64]:
df3[df3["TCAM_15_18"]==max(df3["TCAM_15_18"])] # Vérification si les valeurs inf ont bien été supprimées

,insee,TCAM_15_18,nom,geometry
73,29077,17.81,Guissény,"MULTIPOLYGON (((-4.48600 48.63840, -4.48598 48..."


Pour lire la carte plus facilement, on décide d'ajouter de la couleur en fonction du taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018 (palette nuancée du rouge au vert où le rouge représente un taux négatif et le vert un taux positif).

Pour réaliser cela, on crée une nouvelle colonne, *bins_TCAM1518*, qui segmente et trier les valeurs de données de la colonne *TCAM_15_18* dans des "bins".

In [46]:
# Création colonne bins_TCAM1518
bins_TCAM1518 = pd.IntervalIndex.from_tuples([(-101.0, -50.0), (-50.0, -10.0),
                                              (-10.0, -0.01), (-0.01, 0.01), (0.01, 10.0),
                                              (10.0, 50.0), (50.0, float("inf"))])
                                        # Ici, les valeurs des intervalles sont choisies arbitrairement
    
bins_labels_mapping_TCAM1518 = dict(zip(bins_TCAM1518, range(1, len(bins_TCAM1518) + 1)))

bins_TCAM1518 = pd.cut(df3['TCAM_15_18'],
                    bins=bins_TCAM1518, retbins=False).map(bins_labels_mapping_TCAM1518).astype(float)

df3['bins_TCAM1518'] = bins_TCAM1518

# Création palette
palette_TCAM1518 = {
    'name': 'palette_TCAM1518',
    'type': 'custom',
    'category': 'Custom',
    'colors': ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#ffffbf', '#d9ef8b', '#a6d96a']}#, '#a6d96a', '#66bd63'

In [47]:
df3.head()

,insee,TCAM_15_18,nom,geometry,bins_TCAM1518
1,29002,0.00,Arzano,"POLYGON ((-3.50348 47.89572, -3.50334 47.89597...",4.0
2,29003,2.01,Audierne,"POLYGON ((-4.60262 48.01621, -4.60257 48.01686...",5.0
3,29004,0.00,Bannalec,"POLYGON ((-3.76427 47.92862, -3.76424 47.92912...",4.0
4,29005,0.00,Baye,"POLYGON ((-3.63388 47.84362, -3.63286 47.84428...",4.0
5,29006,-0.83,Bénodet,"POLYGON ((-4.11753 47.87927, -4.11749 47.87936...",3.0


On crée ensuite l'objet cartographie et on ajoute les données à cet objet

In [48]:
carto = KeplerGl()
carto.add_data(data=df3.to_json(), name="Taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


L'objet *carto* doit afficher une carte intéractive.

**A savoir** : Après avoir lancé le chunk suivant, on le met en commentaire et on relance le chunk pour éviter de surcharger le notebook.

In [53]:
# carto

On regarde la configuration par défaut de la cartographie.


**A savoir** : Si le résultat du chunk suivant est un dictionnaire vide, relancer le chunk précédent jusqu'à ce que le dictionnaire ne soit plus vide.

In [50]:
carto.config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': '3v1w0w',
     'type': 'geojson',
     'config': {'dataId': 'Taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018',
      'label': 'Taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018',
      'color': [241, 92, 23],
      'highlightColor': [252, 242, 26, 255],
      'columns': {'geojson': '_geojson'},
      'isVisible': True,
      'visConfig': {'opacity': 0.8,
       'strokeOpacity': 0.8,
       'thickness': 0.5,
       'strokeColor': [34, 63, 154],
       'colorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039

On change la configuration par défaut de la carte pour avoir une visualisation parlante.

In [51]:
tmp_config = carto.config

layers = tmp_config['config']['visState']['layers']
layer_to_update = [layer for layer in layers if layer['config']['dataId'] == "Taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018"]
layer_to_update = layer_to_update[0]

# Pour signifier que le remplissage des communes se fait en fonction de la colonne bins_TCAM1518
layer_to_update["visualChannels"].update({"colorField": {"name": "bins_TCAM1518", "type": "integer"},
                                          "colorScale": "quantize",
                                         }) 

# Pour changer la palette de couleurs remplissant les communes
layer_to_update["config"]["visConfig"]["colorRange"].update(palette_TCAM1518)

# Pour changer l'épaisseur des frontières
layer_to_update["config"]["visConfig"]["thickness"] = 0.2

# Pour changer la couleur des frontières ([0, 0, 0] réprésente la couleur noire en RGB)
layer_to_update["config"]["visConfig"]["strokeColor"] = [0, 0, 0]

carto.config = tmp_config

On enregistre la cartographie au format HTML.

In [52]:
carto.save_to_html(file_name='./Taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018.html')

Map saved to ./Taux de croissance annuel moyen de l’hébergement marchand entre 2015 et 2018.html!


# Sources

1) Données spatiales: découverte de geopandas : https://linogaliana-teaching.netlify.app/geopandas/

2) Wikipédia : https://fr.wikipedia.org/wiki/Saint-G%C3%A9rand

3) Géoconfluences : http://geoconfluences.ens-lyon.fr/glossaire/taux-de-fonction-touristique-doccupation

4) Séries historiques en 2018 : Recensement de la population - Base des principaux indicateurs : https://www.insee.fr/fr/statistiques/5395965#dictionnaire

5) Atout France : Expérimentation nationale sur la réhabilitation de l’immobilier de loisir et la transition énergétique des stations, p22 : https://www.atout-france.fr/sites/default/files/imce/FTI_ImmobilierDeLoisirStation_RapportExperimentation2020.pdf

6) Les hébergements touristiques dans le département de la Manche, p1 : https://www.manche.gouv.fr/content/download/5233/30675/file/note-hebergements-touristiques.pdf

7) Sciences économiques et sociales Lycée Alexandre Dumas – Saint-Cloud : http://seslescours.free.fr/?p=1568

8) Wikipédia : https://fr.wikipedia.org/wiki/Argol_(Finist%C3%A8re)

9) La fonction touristique des territoires : facteur de pression ou de préservation de l'environnement ?, p8 : https://www.statistiques.developpement-durable.gouv.fr/sites/default/files/2018-10/datalab-17-la-fonction-touristique-des-territoires...mars2017a.pdf